# Big Data Exam Report @ UniBo a.y. 2023/2024

- Manuel Andruccioli
- Kelvin Olaiya

In [ ]:
// DO NOT RUN THIS CELL -- ONLY FOR TYPE CHECKER
import org.apache.spark.SparkContext
val sc = new SparkContext("local[*]", "BigDataExam")

In [ ]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [ ]:
sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

## Data structures and definitions

### Utility function for parsing

In [ ]:
def getCharIndexes(line: String, char: Char): Seq[Int] = line.zipWithIndex.filter(_._1 == char).map(_._2) 
def splitAt(s: String, indices: Seq[Int]): Seq[String] = indices match {
  case h +: t => s.splitAt(h) match {
    case (a, b) => a +: splitAt(b, t.map(_ - h))
  }
  case Nil => Seq(s)
}
def parseCSVLine(l: String): Seq[String] = {
  val apices = getCharIndexes(l, '"').grouped(2).map { case Seq(a, b) => (a, b) }.toSeq
  val commas = getCharIndexes(l, ',').filter(i => !apices.exists { case (a, b) => a < i && i < b })
  return splitAt(l, commas).map(_.dropWhile(s => s == ',' || s == ' ')).map(_.replaceAll("^\"|\"$", ""))
}

### Data structures

In [ ]:
case class Track(
  uri: String,
  name: String,
  duration: Int,
  explicit: Boolean,
  artists: String,            // List of artists uri, separated by |
  available_markets: String,  // List of markets, separated by |
  album_uri: String,
  popularity: Int,
)

object Tracks {
  def fromCSVLine(line: String): Option[Track] = 
    parseCSVLine(line) match {
      case Seq(uri, name, duration, explicit, artists, available_markets, album_uri, popularity) =>
        try {
          Some(Track(uri, name, duration.toInt, explicit.toBoolean, artists, available_markets, album_uri, popularity.toInt))
        } catch {
          case _: Throwable => None
        }
    }
}

case class Playlist(
  pid: Int,
  name: String,
  num_follower: Int,
)

object Playlists {
  def fromCSVLine(line: String): Option[Playlist] = 
    parseCSVLine(line) match {
      case Seq(pid, name, num_follower) =>
        try {
            Some(Playlist(pid.toInt, name, num_follower.toInt))
        } catch {
          case _: Throwable => None
        }
    }
}

case class TrackInPlaylist(
  pid: Int,
  track_uri: String,
  pos: Int,
)

object TrackInPlaylists {
  def fromCSVLine(line: String): Option[TrackInPlaylist] = 
    parseCSVLine(line) match {
      case Seq(pid, track_uri, pos) =>
        try {
          Some(TrackInPlaylist(pid.toInt, track_uri, pos.toInt))
        } catch {
          case _: Throwable => None
        }
    }
}

case class Artist(
  uri: String,
  name: String,
  followers: Int,
  genres: String,             // List of genres, separated by |
  popularity: Int,
)

object Artists {
  def fromCSVLine(line: String): Option[Artist] =
    parseCSVLine(line) match {
      case Seq(uri, name, followers, genres, popularity) =>
        try {
          Some(Artist(uri, name, followers.toInt, genres, popularity.toInt))
        } catch {
          case _: Throwable => None
        }
    }
}

case class Album(
  uri: String,
  name: String,
  album_type: String,         // album, compilation, single.
  artists: String,            // List of artists uri, separated by |
  available_markets: String,  // List of markets, separated by |
  release_year: String,
  total_tracks: Int,
)

object Albums {
  def fromCSVLine(line: String): Option[Album] = 
    parseCSVLine(line) match {
      case Seq(uri, name, album_type, artists, available_markets, release_year, total_tracks) =>
        try {
          Some(Album(uri, name, album_type, artists, available_markets, release_year, total_tracks.toInt))
        } catch {
          case _: Throwable => None
        }
    }
}

case class Feature(
  uri: String,
  key: Int,
  loudness: Double,
  tempo: Double,
  mode: Boolean,
  danceability: Double,
  valence: Double,
  instrumentalness: Double,
  liveness: Double,
  acousticness: Double,
  energy: Double,
  speechiness: Double,
)

object Features {
  def fromCSVLine(line: String): Option[Feature] = 
    parseCSVLine(line) match {
      case Seq(uri, key, loudness, tempo, mode, danceability, valence, instrumentalness, liveness, acousticness, energy, speechiness) =>
        try {
          Some(Feature(uri, key.toInt, loudness.toDouble, tempo.toDouble, mode.toInt == 1, danceability.toDouble, valence.toDouble, instrumentalness.toDouble, liveness.toDouble, acousticness.toDouble, energy.toDouble, speechiness.toDouble))
        } catch {
          case e: Throwable => None
        }
    }
}

## Dataset exploration

In [ ]:
val datasetPath = "dataset/"
val outputPath = s"${datasetPath}output/"

val albumRdd = sc.textFile(s"${datasetPath}albums.csv").flatMap(Albums.fromCSVLine)
val artistRdd = sc.textFile(s"${datasetPath}artists.csv").flatMap(Artists.fromCSVLine)
val featureRdd = sc.textFile(s"${datasetPath}features.csv").flatMap(Features.fromCSVLine)
val playlistRdd = sc.textFile(s"${datasetPath}playlists.csv").flatMap(Playlists.fromCSVLine)
val trackInPlaylistRdd = sc.textFile(s"${datasetPath}tracks_in_playlists.csv").flatMap(TrackInPlaylists.fromCSVLine)
val trackRdd = sc.textFile(s"${datasetPath}tracks.csv").flatMap(Tracks.fromCSVLine)

In [ ]:
val albumRddCached = albumRdd.cache()
val artistRddCached = artistRdd.cache()
val featureRddCached = featureRdd.cache()
val playlistRddCached = playlistRdd.cache()
val trackInPlaylistRddCached = trackInPlaylistRdd.cache()
val trackRddCached = trackRdd.cache()

In [ ]:
println(s"Number of Albums: ${albumRddCached.count()}")
println(s"Number of Artists: ${artistRddCached.count()}")
println(s"Number of Track's Feature: ${featureRddCached.count()}")
println(s"Number of Playlist: ${playlistRddCached.count()}")
println(s"Number of Tracks add in Playlists: ${trackInPlaylistRddCached.count()}")
println(s"Number of Tracks: ${trackRddCached.count()}")

### Average number of tracks

In [ ]:
println(s"In Playlist: ${(trackInPlaylistRddCached.count().toDouble / playlistRddCached.count().toDouble).round}")
println(s"In Album: ${(albumRddCached.map(_.total_tracks).sum() / albumRddCached.count().toDouble).round}")
println(s"Per Artist (double counting on artist): ${
  trackRddCached.flatMap(t => t.artists.split('|').map(a => (a, 1))).
    reduceByKey(_ + _).
    map { case (_, count) => count }.
    mean().round}")

### Explicit vs Non-Explicit Tracks

In [ ]:
import org.apache.spark.sql.SaveMode

trackRddCached.map(t => (t.explicit, 1)).
  reduceByKey(_ + _).
  coalesce(1).
  toDF().write.format("csv").mode(SaveMode.Overwrite).save(s"${outputPath}explicit_vs_non_explicit.csv")

![Explicit vs Non-Explicit Tracks](./img/explicit_vs_non_explicit.png)

### Tracks per year

In [ ]:
albumRddCached.map(a => a.release_year).
  distinct().collect().sorted

albumRddCached.map(a => (a.uri, a.release_year)).
  join(trackRddCached.map(t => (t.album_uri, (t.uri, t.popularity)))).
  map { case (albumUri, (releaseYear, (trackUri, popularity))) => (releaseYear, 1) }.
  reduceByKey(_ + _).
  coalesce(1).
  sortByKey().
  toDF().write.format("csv").mode(SaveMode.Overwrite).save(s"${outputPath}tracks_per_year.csv")

![Track per Year](./img/tracks_per_year.png)

### Top

#### Most followed Playlist

In [ ]:
playlistRddCached.map(p => (p.pid, p.name, p.num_follower)).
  sortBy(_._3, ascending = false).
  coalesce(1).
  toDF().write.format("csv").mode(SaveMode.Overwrite).save(s"${outputPath}playlist_with_followers.csv")

![Top 10 playlist per followers](./img/top10_playlist_per_followers.png)

#### Most popular Playlist

The playlist popularity is calculated as the mean of the popularity of the tracks in the playlist.

In [ ]:
trackInPlaylistRddCached.
  map(t => (t.track_uri, t.pid)).
  join(trackRddCached.map(t => (t.uri, t.popularity)))
  .map { case (_, (pid, popularity)) => (pid, popularity) }.
  aggregateByKey((0, 0))(
    { case ((acc, count), popularity) => (acc + popularity, count + 1) },
    { case ((acc1, count1), (acc2, count2)) => (acc1 + acc2, count1 + count2) }
  ).mapValues { case (acc, count) => acc.toDouble / count }.
  join(playlistRddCached.map(p => (p.pid, p.name))).
  map { case (pid, (popularity, name)) => (pid, name, popularity) }.
  sortBy(_._3, ascending = false).
  coalesce(1).
  toDF().write.format("csv").mode(SaveMode.Overwrite).save(s"${outputPath}playlist_with_popularity.csv")

![Top playlist per popularity](./img/top_playlist_per_popularity.png)

In [ ]:
sc.getPersistentRDDs.foreach(_._2.unpersist())

## Job 1

Given the following metrics:

  - track's popularity
  - average popularity of tracks in year
  - artist popularity (average if more artists are present in the track)

Understand which metrics influence mostly the playlists, averaging the values of the tracks appearing in each.
Also, aggregate the playlists on the previously calculated influence by averaging for the number of playlist followers.

> **most influent**: the metric that has the highest average value for a playlist. 

The query let us answering the following question:
a playlist influenced most by the popularity of the tracks has, on average, 500 followers. (Same for the other two metrics).

### Non optimized

First of all, we need to calculate the `popularity of the year`.

$$ \texttt{Popularity of the year} = 
\frac{
  \sum{\texttt{popularity of tracks in that year}}
}{
  \texttt{Number of tracks in that year}
}
$$

Given the fact that the `release year` of a track is on the album, we need to join the track with the album to get it.
After this first step, we can calculate the average popularity of the year.

In [ ]:
val trackWithAlbumUriAsKey = trackRdd.map(t => (t.album_uri, (t.uri, t.popularity)))

// year -> popularity of the year
val popPerYear = albumRdd.map(a => (a.uri, a.release_year)).
  join(trackWithAlbumUriAsKey).
  map { case (albumUri, (releaseYear, (trackUri, popularity))) => (releaseYear, popularity) }.
  aggregateByKey((0, 0))(
    { case ((acc, count), popularity) => (acc + popularity, count + 1) },
    { case ((acc1, count1), (acc2, count2)) => (acc1 + acc2, count1 + count2) }
  ).mapValues { case (acc, count) => acc.toDouble / count }

After that, we need to get the `artist's popularity of a track`.
Given the fact that a track can have multiple artists, the `artist's popularity of a track` will be a mean of the `artist popularity` of the artists in the track.

$$ \texttt{Artist's popularity of a track} =
\frac{
  \sum{\texttt{popularity of artists in the track}}
}{
  \texttt{Number of artists in the track}
}
$$

> **Note**: The `artists_uri` is a denormalized field in the track, so we need to split it.

In [ ]:
val artistWithPopularity = artistRdd.map(a => (a.uri, a.popularity))

// track -> artist's popularity
val trackWithArtistPopularity = trackRdd.flatMap(t => t.artists.split('|').map(artistUri => (artistUri, t.uri))).
  join(artistWithPopularity).
  map { case (artistUri, (trackUri, artistPopularity)) => (trackUri, artistPopularity) }.
  aggregateByKey((0, 0))(
    { case ((acc, count), popularity) => (acc + popularity, count + 1) },
    { case ((acc1, count1), (acc2, count2)) => (acc1 + acc2, count1 + count2) }
  ).mapValues { case (acc, count) => acc.toDouble / count }

At this point we can join all the data to get the `popularity`, `average popularity of the year` and `artist's popularity` of a track.

$$
\texttt{track_uri} \rightarrow (\texttt{popularity}, \texttt{average popularity of the year}, \texttt{artist's popularity})
$$

After that, we can join the `track` with the `playlist` and perform the aggregation, achieving the average of the metrics for each playlist:

$$
\texttt{pid} \rightarrow (\texttt{popularity}, \texttt{average popularity of the year}, \texttt{artist's popularity})
$$

Now, we can calculate the `most influent` metric for each playlist taking the maximum value of the metrics.
  
Finally, we can aggregate the playlists by the `most influent` metric and calculate the average number of followers for each metric.

In [ ]:
trackWithAlbumUriAsKey.
  join(albumRdd.map(a => (a.uri, a.release_year))).
  map { case (albumUri, ((trackUri, popularity), releaseYear)) => (releaseYear, (trackUri, popularity)) }.
  join(popPerYear).
  map { case (releaseYear, ((trackUri, popularity), avgPopularityInYear)) => (trackUri, (popularity, avgPopularityInYear)) }.
  join(trackWithArtistPopularity).
  map { case (trackUri, ((popularity, avgPopularityInYear), avgArtistPopularity)) => (trackUri, (popularity, avgPopularityInYear, avgArtistPopularity)) }.
  join(trackInPlaylistRdd.map(t => (t.track_uri, t.pid))).
  map {
      case (trackUri, ((popularity, avgPopularityInYear, avgArtistPopularity), pid)) => (pid, (popularity, avgPopularityInYear, avgArtistPopularity))
  }.
  aggregateByKey((0.0, 0.0, 0.0, 0))(
    { case ((accPop, accAvgPopInYear, accAvgArtistPop, count), (popularity, avgPopularityInYear, avgArtistPopularity)) =>  
        (accPop + popularity, accAvgPopInYear + avgPopularityInYear, accAvgArtistPop + avgArtistPopularity, count + 1)
    },
    { case ((accPop1, accAvgPopInYear1, accAvgArtistPop1, count1), (accPop2, accAvgPopInYear2, accAvgArtistPop2, count2)) =>
      (accPop1 + accPop2, accAvgPopInYear1 + accAvgPopInYear2, accAvgArtistPop1 + accAvgArtistPop2, count1 + count2)
    }
  ).
  mapValues { case (accPop, accAvgPopInYear, accAvgArtistPop, count) => (accPop / count, accAvgPopInYear / count, accAvgArtistPop / count) }.
  map {
      case (pid, (avgPop, avgPopInYear, avgArtistPop)) =>
        val maxAvg = Math.max(avgPop, Math.max(avgPopInYear, avgArtistPop))
        val indexOfBestAvg = Seq(avgPop, avgPopInYear, avgArtistPop).indexWhere(_ >= maxAvg)
        (pid, indexOfBestAvg)
  }.
  join(playlistRdd.map(p => (p.pid, p.num_follower))).
  map { case (pid, (indexOfBestAvg, numFollower)) => (indexOfBestAvg, (numFollower, 1)) }.
  reduceByKey { case ((accFollowers1, c1), (accFollowers2, c2)) => (accFollowers1 + accFollowers2, c1 + c2) }.
  mapValues { case (accF, c) => accF / c }.
  collect.foreach(println)

### Optimization

## Job 2

Given the following classes: slowly danceable (tempo <= 130BPM, danceability > 0.5), swiftly danceable (tempo >130BPM, danceability > 0.5), slowly undanceable (tempo <= 130BPM, danceability <= 0.5), swiftly undanceable (tempo >130BPM, danceability <= 0.5); and the various keys (C, C#/Db, ...).
  for each class and (key ---OR--- range of followers) get:
  - The number of playlist.
  - Average playlist's explicitness percentage.
  - Average number of tracks in playlist.
  - Average number of playlist followers.
  <!-- - Average playlist danceability.
  - Average playlist tempo. -->
  (The key of a playlist is the most present key among its tracks)

In [ ]:
def toClass(tempo: Double, danceablility: Double): String = (tempo, danceablility) match {
  case (t, d) if t <= 130 && d > 0.5 => "slowly danceable"
  case (t, d) if t > 130 && d > 0.5 => "swiftly danceable"
  case (t, d) if t <= 130 && d <= 0.5 => "slowly undanceable"
  case (t, d) if t > 130 && d <= 0.5 => "swiftly undanceable"
}

def joinMap(map1: Map[Int, Int], map2: Map[Int, Int]): Map[Int, Int] = map1.map { case(k, v) => (k, map2.getOrElse(k, 0) + v) }
def incrementOnKey(map: Map[Int, Int], key: Int) = {
  val current = map.getOrElse(key, 0)
  map.updated(key, current + 1)
}

val features = featureRdd.map(t => (t.uri, (t.tempo, t.danceability, t.key))).
  join(trackRdd.map(t => (t.uri, t.explicit))).
  map { case (uri, ((t, d, k), e)) => (uri, (t, d, k, e)) }

val trackInPlaylistWithFeatures = trackInPlaylistRdd.map(t => (t.track_uri, t.pid)).join(features)

val playlistClasses = trackInPlaylistWithFeatures.
        map { case (trackUri, (pid, (t, d, k, e))) => (pid, (t, d, k, e)) }.
        aggregateByKey((0.0, 0.0, (0 to 11).map((_, 0)).toMap, 0.0, 0))(
          { case ((accT, accD, ks, ec, c), (t, d, k, e)) => (accT+t, accD+d, incrementOnKey(ks, k), ec+(if (e) 1 else 0), c+1) },
          { case ((accT1, accD1, k1, ec1, c1), (accT2, accD2, k2, ec2, c2)) => (accT1+accT2, accD1+accD2, joinMap(k1, k2), ec1+ec2, c1+c2) }).
        mapValues({ case (accT, accD, k, ec, c) => (accT/c, accD/c, k.maxBy(_._2)._1, ec/c, c) }).
        map { case (pid, (avgT, avgD, k, avgE, c)) => (pid, (k, toClass(avgT, avgD), avgE, c)) }// (pid, (k, class, avgE, c))

playlistRdd.map(p => (p.pid, p.num_follower)).join(playlistClasses). // (pid, (num_follower, (k, class, avgE, c)))
        map { case (pid, (num_follower, (k, cls, avgE, tc))) => ((k, cls), (num_follower, avgE, tc)) }.
        aggregateByKey((0.0, 0.0, 0.0, 0))(
          { case ((accF, accE, accTC, c), (f, e, tc)) => (accF+f, accE+e, accTC+tc, c+1) },
          { case ((accF1, accE1, accTC1, c1), (accF2, accE2, accTC2, c2)) => (accF1+accF2, accE1+accE2, accTC1+accTC2, c1+c2) }
        ).
        mapValues { case (accF, accE, accTC,c) => (accF/c, accE/c, accTC/c, c) }. // ((k, class), (avgF, avgE, avgTC, c))
        collect.foreach(println)
